<a href="https://colab.research.google.com/github/hangluongdatascience/hangluongdatascience/blob/main/kol_text_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Import Library**:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install that-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [5]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from that_nlp_library.text_main import *
from that_nlp_library.utils import seed_everything

In [6]:
from underthesea import text_normalize
from functools import partial
from pathlib import Path
from transformers import DataCollatorWithPadding,RobertaTokenizer
from transformers.models.roberta.modeling_roberta import RobertaForSequenceClassification
# import nlpaug.augmenter.char as nac
from datasets import load_dataset
import random
import pandas as pd
import numpy as np
from datasets import load_dataset
from importlib.machinery import SourceFileLoader

**2. Load data**

In [7]:
PATH = Path('/content/drive/MyDrive/kol_prediction')

In [8]:
PATH/'kol_text_classifi'

PosixPath('/content/drive/MyDrive/kol_prediction/kol_text_classifi')

In [9]:
df = pd.read_csv(PATH/'kol_text_classifi'/'kol_group_post.csv',encoding='utf-8-sig')

In [10]:
df = df[~df.L3.isna()].reset_index(drop=True)

In [11]:
df.shape

(297, 18)

**3. Load model**

In [12]:
#process xử lý text (clean up, filter ,trim, label processing,....)
tdc = TextDataController.from_df(df,
                         main_text='content',
                         label_names='L3',
                         sup_types='classification',
                         content_transformations=[text_normalize,str.lower],
                         val_ratio=0.2,
                         stratify_cols='L3',
                         batch_size=100,
                         seed=42,
                         num_proc=1,
                         verbose=True
                        )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
post_id        183
user_name      183
user_id        183
post_date      183
link           183
link_mobile    183
comments       183
reactions      183
hashtag        201
group_id       183
import_date    183
Sentiment      183
Post owner     183
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 1 rows


In [13]:
cache_dir=PATH/'envibert_tokenizer'
tokenizer = SourceFileLoader("envibert.tokenizer",  str(cache_dir/'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)

In [14]:
#độ dài tối đa của 'content'
tdc.process_and_tokenize(tokenizer,max_length=140,shuffle_trn=True)


-------------------- Start Main Text Processing --------------------
----- Label Encoding -----


Map:   0%|          | 0/297 [00:00<?, ? examples/s]

Done
-------------------- Text Transformation --------------------
----- text_normalize -----


Map:   0%|          | 0/297 [00:00<?, ? examples/s]

----- lower -----


...

Flattening the indices:   0%|          | 0/235 [00:00<?, ? examples/s]

Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/235 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Done


In [20]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['content', 'L3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 235
    })
    validation: Dataset({
        features: ['content', 'L3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 60
    })
})

In [21]:
print(pd.Series(tdc.main_ddict['train']['L3']).nunique())
pd.Series(tdc.main_ddict['train']['L3']).value_counts()

17


Mission                    37
Webinar                    35
Payment / Commission       28
Account / Tracking link    22
Q&A                        22
Livestream                 20
Shopee Video               14
Referral / Ambassador      11
Campaign Push               8
Deal updates                7
Upsized commission          6
Ads sponsorship             6
Fun topics                  5
Group policy                4
Welcome Voucher             4
Minigame                    3
KOC Promotion               3
dtype: int64

In [22]:
print(pd.Series(tdc.main_ddict['validation']['L3']).nunique())
pd.Series(tdc.main_ddict['validation']['L3']).value_counts()


17


Mission                    9
Webinar                    9
Payment / Commission       7
Account / Tracking link    6
Q&A                        6
Livestream                 5
Referral / Ambassador      3
Shopee Video               3
Upsized commission         2
Campaign Push              2
Deal updates               2
Welcome Voucher            1
KOC Promotion              1
Minigame                   1
Fun topics                 1
Group policy               1
Ads sponsorship            1
dtype: int64

**4. Train model**

In [15]:
from transformers.models.roberta.modeling_roberta import RobertaModel
from that_nlp_library.models.roberta.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import f1_score, accuracy_score

In [16]:
num_classes = len(tdc.label_lists[0])

In [17]:
model_name='nguyenvulebinh/envibert'
roberta_body = RobertaModel.from_pretrained(model_name)

(…)lebinh/envibert/resolve/main/config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/283M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at nguyenvulebinh/envibert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# model_name='nguyenvulebinh/envibert'
# roberta_body = RobertaModel.from_pretrained(model_name)

# _model_kwargs={
#     # overall model hyperparams
#     'head_class_sizes':num_classes,
#     'head_class': ConcatHeadSimple,
#     # classfication head hyperparams
#     'layer2concat':1,
#     'classifier_dropout':0.01
# }

# model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
#                                   cpoint_path = model_name,
#                                   output_hidden_states=True,
#                                   seed=42,
#                                   body_model=roberta_body,
#                                   model_kwargs = _model_kwargs)

# metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
# controller = ModelController(model,tdc,seed=42)

In [24]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

0


RuntimeError: ignored

In [23]:
pd.Series(tdc.main_ddict['train']['L3']).value_counts(normalize=True)

Mission                    0.157447
Webinar                    0.148936
Payment / Commission       0.119149
Account / Tracking link    0.093617
Q&A                        0.093617
Livestream                 0.085106
Shopee Video               0.059574
Referral / Ambassador      0.046809
Campaign Push              0.034043
Deal updates               0.029787
Upsized commission         0.025532
Ads sponsorship            0.025532
Fun topics                 0.021277
Group policy               0.017021
Welcome Voucher            0.017021
Minigame                   0.012766
KOC Promotion              0.012766
dtype: float64

**5. Turn model**

In [19]:
model_name='nguyenvulebinh/envibert'
roberta_body = RobertaModel.from_pretrained(model_name)
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)

metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 1e-4
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )
# Epoch	Training Loss	Validation Loss	F1 Score L3	Accuracy Score L3
# 1	No log	2.642582	0.529650	0.616667
# 2	0.099600	2.622562	0.514356	0.633333
# 3	0.099600	2.906531	0.555392	0.633333
# 4	0.098400	2.834615	0.598127	0.650000
# 5	0.098400	2.664778	0.633258	0.650000
# 6	0.024800	2.597922	0.614539	0.650000
# 7	0.024800	2.757308	0.650135	0.650000
# 8	0.003800	2.759731	0.650135	0.650000

# 6	1.222800	1.577971	0.405799	0.566667
# 7	1.222800	1.533020	0.466790	0.600000
# 8	0.702200	1.537630	0.462416	0.600000

Some weights of RobertaModel were not initialized from the model checkpoint at nguyenvulebinh/envibert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading body weights. This assumes the body is the very first block of your custom architecture
Total parameters: 70728977
Total trainable parameters: 70728977


ValueError: ignored

In [23]:
controller.trainer.model.save_pretrained(PATH/'first_run_0.462_lr_1e-4_8_epoch')

In [27]:
model_name=PATH/'first_run_0.462_lr_1e-4_8_epoch'
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 9e-5
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

Some weights of the model checkpoint at /content/drive/MyDrive/kol_prediction/first_run_0.462_lr_1e-4_8_epoch were not used when initializing RobertaHiddenStateConcatForSequenceClassification: ['body_model.pooler.dense.bias', 'body_model.pooler.dense.weight']
- This IS expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total parameters: 70138385
Total trainable parameters: 70138385


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L3,Accuracy Score L3
1,No log,1.630743,0.473979,0.600000
2,0.511700,1.770552,0.445436,0.600000
3,0.511700,1.731057,0.492446,0.583333
4,0.161800,1.786767,0.534704,0.650000
5,0.161800,1.865128,0.580944,0.633333
6,0.026500,1.867682,0.582790,0.650000
7,0.026500,1.878207,0.573686,0.633333
8,0.018800,1.883375,0.538392,0.616667


In [28]:
controller.trainer.model.save_pretrained(PATH/'second_run_0.538_lr_9e-5_8_epoch')

In [28]:
model_name=PATH/'first_run_0.462_lr_1e-4_8_epoch'
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 9e-5
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )


# Epoch	Training Loss	Validation Loss	F1 Score L3	Accuracy Score L3
# 1	No log	1.630743	0.473979	0.600000
# 2	0.511700	1.770552	0.445436	0.600000
# 3	0.511700	1.731057	0.492446	0.583333
# 4	0.161800	1.786767	0.534704	0.650000
# 5	0.161800	1.865128	0.580944	0.633333
# 6	0.026500	1.867682	0.582790	0.650000
# 7	0.026500	1.878207	0.573686	0.633333
# 8	0.018800	1.883375	0.538392	0.616667

Some weights of the model checkpoint at /content/drive/MyDrive/kol_prediction/first_run_0.462_lr_1e-4_8_epoch were not used when initializing RobertaHiddenStateConcatForSequenceClassification: ['body_model.pooler.dense.weight', 'body_model.pooler.dense.bias']
- This IS expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total parameters: 70138385
Total trainable parameters: 70138385


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L3,Accuracy Score L3
1,No log,1.630743,0.473979,0.600000
2,0.511700,1.770552,0.445436,0.600000
3,0.511700,1.731057,0.492446,0.583333
4,0.161800,1.786767,0.534704,0.650000
5,0.161800,1.865128,0.580944,0.633333
6,0.026500,1.867682,0.582790,0.650000
7,0.026500,1.878207,0.573686,0.633333
8,0.018800,1.883375,0.538392,0.616667


In [29]:
controller.trainer.model.save_pretrained(PATH/'second_run_0.538_lr_9e-5_8_epoch')

In [30]:
model_name=PATH/'first_run_0.462_lr_1e-4_8_epoch'
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 1.1e-4
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

Some weights of the model checkpoint at /content/drive/MyDrive/kol_prediction/first_run_0.462_lr_1e-4_8_epoch were not used when initializing RobertaHiddenStateConcatForSequenceClassification: ['body_model.pooler.dense.weight', 'body_model.pooler.dense.bias']
- This IS expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total parameters: 70138385
Total trainable parameters: 70138385


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L3,Accuracy Score L3
1,No log,1.704549,0.422473,0.533333
2,0.530400,1.768307,0.498274,0.616667
3,0.530400,1.883958,0.461830,0.566667
4,0.144800,1.739838,0.565168,0.650000
5,0.144800,1.968488,0.525660,0.616667
6,0.022100,1.925806,0.543513,0.666667
7,0.022100,1.938120,0.535756,0.650000
8,0.010800,1.942865,0.535756,0.650000


In [24]:
model_name=PATH/'first_run_0.462_lr_1e-4_8_epoch'
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.1
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 1e-4
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

Some weights of the model checkpoint at /content/drive/MyDrive/kol_prediction/first_run_0.462_lr_1e-4_8_epoch were not used when initializing RobertaHiddenStateConcatForSequenceClassification: ['body_model.pooler.dense.bias', 'body_model.pooler.dense.weight']
- This IS expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total parameters: 70138385
Total trainable parameters: 70138385


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L3,Accuracy Score L3
1,No log,1.665561,0.388206,0.566667
2,0.522800,1.808289,0.494092,0.600000
3,0.522800,1.698381,0.503756,0.633333
4,0.161500,1.756690,0.544453,0.666667
5,0.161500,1.968084,0.547165,0.616667
6,0.023400,1.944056,0.538392,0.616667
7,0.023400,1.950353,0.538392,0.616667
8,0.014700,1.955504,0.538392,0.616667


In [26]:
model_name='nguyenvulebinh/envibert'
roberta_body = RobertaModel.from_pretrained(model_name)
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)

metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 9.5e-5
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

Some weights of RobertaModel were not initialized from the model checkpoint at nguyenvulebinh/envibert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading body weights. This assumes the body is the very first block of your custom architecture
Total parameters: 70728977
Total trainable parameters: 70728977


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L3,Accuracy Score L3
1,No log,2.672331,0.038755,0.183333
2,2.644300,2.413900,0.073647,0.300000
3,2.644300,2.038501,0.169282,0.366667
4,1.930200,1.816628,0.198226,0.416667
5,1.930200,1.544584,0.331669,0.516667
6,0.930800,1.529739,0.426674,0.533333
7,0.930800,1.491385,0.468117,0.583333
8,0.512200,1.495375,0.468117,0.583333


In [32]:
model_name=PATH/'second_run_0.538_lr_9e-5_8_epoch'
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 9e-5
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

Total parameters: 70138385
Total trainable parameters: 70138385


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L3,Accuracy Score L3
1,No log,2.035550,0.544744,0.616667
2,0.011100,2.353551,0.509410,0.616667
3,0.011100,2.343330,0.485691,0.616667
4,0.055700,2.798339,0.528105,0.600000
5,0.055700,2.525594,0.505345,0.583333
6,0.002000,2.467197,0.570691,0.616667
7,0.002000,2.465226,0.570691,0.616667
8,0.001400,2.465281,0.570691,0.616667


In [33]:
controller.trainer.model.save_pretrained(PATH/'third_run_0.57_lr_9e-5_8_epoch')

In [29]:
model_name=PATH/'second_run_0.538_lr_9e-5_8_epoch'
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 1e-4
bs=2
wd=0.01
epochs= 8

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

Total parameters: 70138385
Total trainable parameters: 70138385


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score L3,Accuracy Score L3
1,No log,2.038829,0.480923,0.600000
2,0.016700,2.530063,0.481081,0.533333
3,0.016700,2.507083,0.579056,0.616667
4,0.025000,2.979481,0.541958,0.616667
5,0.025000,2.592499,0.561080,0.633333
6,0.005800,2.612245,0.592484,0.633333
7,0.005800,2.629797,0.599481,0.650000
8,0.001300,2.625862,0.599481,0.650000


In [34]:
controller.trainer.model.save_pretrained(PATH/'third_run_0.599_lr_9e-5_8_epoch')

In [2]:
model_name=PATH/'third_run_0.599_lr_9e-5_8_epoch'
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':2,
    'classifier_dropout':0.01
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name,
                                  output_hidden_states=True,
                                  seed=42,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=42)

seed_everything(42)

lr = 1e-4
bs=2
wd=0.01
epochs= 10

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

NameError: ignored